# Analyse effectif

In [ ]:
from CTG_Utils.CTGClasses import EffectifCtg
year = 2024

effectif_n = EffectifCtg(year-1)
effectif_n_plus1 = EffectifCtg(year)
nouveaux_licenciés = set(effectif_n_plus1.effectif["N° Licencié"])-set(effectif_n.effectif["N° Licencié"])
licencies_non_renouvelles = set(effectif_n.effectif["N° Licencié"]) - set(effectif_n_plus1.effectif["N° Licencié"])
moy_age_entrants = effectif_n_plus1.effectif[effectif_n_plus1.effectif['N° Licencié'].isin(nouveaux_licenciés)]['Age'].mean() + 1
moy_age_sortants = effectif_n.effectif[effectif_n.effectif['N° Licencié'].isin(licencies_non_renouvelles)]['Age'].mean()
print(f"{len(nouveaux_licenciés)} nouveaux licenciés de moyenne d'âge de {round(moy_age_entrants,1)}")
print(f"{len(licencies_non_renouvelles)} licences non renouvellées de moyenne d'âge de {round(moy_age_sortants,1)}")
print()

effectif = EffectifCtg(year)
effectif.stat()
effectif.plot_histo()



In [ ]:
# présence au club

import os
import re
from pathlib import Path

import pandas as pd

import CTG_Utils as ctg

list_date = [int(x) for x in os.listdir(ctg.GLOBAL['ROOT']) if re.findall('^\d{4}$',x)]
list_df = []

for date in list_date:
    df = pd.read_excel(Path(ctg.GLOBAL['ROOT']) /Path(str(date)) / Path('DATA') / Path(str(date)+'.xlsx'))
    df['date'] = date
    list_df.append(df[['N° Licencié','Nom','Prénom','date']])
    
df = pd.concat(list_df) #.to_excel(file / Path('effectif_total.xlsx'),index=False)

dic = {}
list_num_licence = []
list_nom = []
list_prenom = []
list_date_ = []
for licence in df.groupby('N° Licencié'):
    list_c = [x if x in licence[1]['date'].to_list() else None for x in list_date]
    #if len(list_c) - list_c.count(None) == 1: singleton
    list_num_licence.append(licence[0])
    list_nom.append(licence[1]['Nom'].unique()[0])
    list_prenom.append(licence[1]['Prénom'].unique()[0])
    list_date_.append(list_c)

    
dic['N° Licencié'] = list_num_licence
dic['Nom'] = list_nom
dic['Prénom'] = list_prenom
dic['date'] = list_date_

df = pd.DataFrame.from_dict(dic)
split_df = pd.DataFrame(df['date'].tolist(), columns=list_date)

df = pd.concat([df, split_df], axis=1)
df = df.drop('date',axis=1)
out_path = Path(ctg.GLOBAL['ROOT']) / Path(str(list_date[-1])) / Path('DATA') / Path('effectif_total.xlsx')
df.to_excel(out_path)

# Evolution des effectifs

In [ ]:
from CTG_Utils.CTG_effectif import evolution_effectif

evolution_effectif()

# Evolution de l'âge moyen CTG

In [ ]:
from CTG_Utils.CTG_effectif import evolution_age_median
evolution_age_median()

# Tracé localisation géographique des adhérents CTG


In [ ]:
from pathlib import Path

import pandas as pd

import CTG_Utils as ctg

year = 2024

df_effectif = pd.read_excel(ctg.GLOBAL['ROOT'] / Path(str(year)) / Path('DATA')/ Path(str(year)+'.xlsx'))
kol = ctg.plot_ctg(df_effectif)

kol.save(Path(ctg.GLOBAL["ROOT"]) / Path(str(year)) / Path('STATISTIQUES') / Path("ctg.html"))
kol

# Statistique VAE

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

# Internal import
import CTG_Utils as ctg
# function to add value labels
def addlabels(x,y):
    for i in range(1,len(x)):
        plt.text(x[i]-0.2,y[i]+0.5,y[i],size=10)

courant_year = 2024

last_year = 2018
years =[]
nb_vae_m = []
nb_vae_f = []
nb_vae_tot = []
for year in range(last_year,courant_year+1):
    df_N1 = pd.read_excel(Path(ctg.GLOBAL['ROOT']) / Path(str(year)) / Path('DATA') /Path(str(year)+'.xlsx'))
    years.append(year)
    nb_vae_m.append(sum((df_N1['Pratique VAE'] == 'Oui') & (df_N1['Sexe'] == 'M')))
    nb_vae_f.append(sum((df_N1['Pratique VAE'] == 'Oui') & (df_N1['Sexe'] == 'F')))
    nb_vae_tot.append(sum((df_N1['Pratique VAE'] == 'Oui') & (df_N1['Sexe'] == 'F')) +
                      sum((df_N1['Pratique VAE'] == 'Oui') & (df_N1['Sexe'] == 'M')))
plt.bar(years, nb_vae_m,label= 'Homme')
plt.bar(years, nb_vae_f,bottom=nb_vae_m,label= 'Femme')
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.ylabel('nombre de VAEs',size=15)
plt.legend()
plt.ylabel('nombre de VAE')
addlabels(years, [x+y for x,y in zip(nb_vae_m,nb_vae_f)])
print(nb_vae_tot)   

In [ ]:
# Standard library imports
from pathlib import Path
import datetime

# 3rd party imports
import numpy as np
import pandas as pd
import yaml

# Internal import
from CTG_Utils.CTG_config import _config_ctg
GLOBAL = _config_ctg()

class effectif_corrige_ctg():
    
    '''Lecture du fichier effectif et correction
    '''   
        
    def __init__(self,year=None):
    
        if year is not None:
            file_effectif = str(year) + '.xlsx'
        else:
            currentDateTime = datetime.datetime.now()
            date = currentDateTime.date()
            year = date.strftime("%Y")
            file_effectif = year + '.xlsx'
            
        self.year = year

        self.df_effectif = pd.read_excel(GLOBAL['ROOT']/ Path(str(self.year))/Path('DATA')/ Path(file_effectif))
        self.df_effectif = self.df_effectif[['N° Licencié', 'Nom','Prénom','Sexe','Pratique VAE']]

        self.correction_effectif()
        
        for num_licence in self.dic_correction_licence.keys():
            idx = self.df_effectif[self.df_effectif["N° Licencié"] == num_licence].index
            self.df_effectif.loc[idx,'Prénom'] = self.dic_correction_licence[num_licence]['Prénom']
            self.df_effectif.loc[idx,'Nom'] = self.dic_correction_licence[num_licence]['Nom']

        prenom, nom, sexe = zip(*self.part_club)
        part_club_dic = {'N° Licencié':list(range(len(self.part_club))), 'Prénom':prenom,'Nom':nom,'Sexe':sexe}
        df_part_club = pd.DataFrame.from_dict(part_club_dic)
        self.df_effectif = pd.concat([self.df_effectif, df_part_club], ignore_index=True, axis=0)

        if self.list_non_licencie:
            prenom, nom, sexe = zip(*self.list_non_licencie)
            dict_non_licencie = {'N° Licencié':np.array(range(len(nom)))+10, 'Prénom':prenom,'Nom':nom,'Sexe':sexe}

            df_non_licencie = pd.DataFrame.from_dict(dict_non_licencie)
            self.df_effectif = pd.concat([df_effectif, df_non_licencie], ignore_index=True, axis=0)

        self.df_effectif['Prénom1'] = self.df_effectif['Prénom'].str[0]
        self.df_effectif['Prénom'] = self.df_effectif['Prénom'].str.replace(' ','-')
        
    def correction_effectif(self):
        
        '''Reads the yaml file located in ROOT/year/DATA 
        '''
        path_cor_yaml = GLOBAL['ROOT'] / Path(str(self.year)) / Path('DATA') / Path('CTG_correction.yaml')

        with open(path_cor_yaml, "r",encoding='utf8') as stream:
            data_list_dict = yaml.safe_load(stream)

        self.list_non_licencie  = []   
        if data_list_dict['list_non_licencie']:
            self.list_non_licencie = [(x.split(',')[0].strip(),x.split(',')[1].strip(),x.split(',')[2].strip())
                                 for x in data_list_dict['list_non_licencie']]
        self.part_club = [(x.split(',')[0].strip(),x.split(',')[1].strip(),x.split(',')[2].strip()) 
                             for x in data_list_dict['dic_part_club']]
        self.dic_correction_licence = data_list_dict['dic_correction_licence']
        self.dic_correction_licence = {list(x.keys())[0] : list(x.values())[0] for x in self.dic_correction_licence}
           